## Analyzing IPL Matches

In [ ]:
# File location and type
file_location1 = "/FileStore/tables/ipl_ball_by_ball.csv"
file_location2 = "/FileStore/tables/ipl_matches.csv"
file_location3 = "/FileStore/tables/ipl_venue.csv"
file_type = "csv"

In [ ]:
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df1 = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location1)

df1.show(5)

+--------+------+-----+----+-------------+-------------+------------+------------+----------+----------+------------+---------+--------------+----------------+-------+-----------+--------------------+--------------+
|match_id|inning|overs|ball|      batsman|  non_striker|      bowler|batsman_runs|extra_runs|total_runs|non_boundary|is_wicket|dismissal_kind|player_dismissed|fielder|extras_type|        batting_team|  bowling_team|
+--------+------+-----+----+-------------+-------------+------------+------------+----------+----------+------------+---------+--------------+----------------+-------+-----------+--------------------+--------------+
|  419157|     2|   16|   4|      V Kohli|R Vinay Kumar|CRD Fernando|           0|         0|         0|           0|        0|            NA|              NA|     NA|         NA|Royal Challengers...|Mumbai Indians|
|  419157|     2|   16|   5|      V Kohli|R Vinay Kumar|CRD Fernando|           1|         0|         1|           0|        0|         

In [ ]:
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df2 = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location2)

df2.show(5)

+--------+-------------------+---------------+--------+-------------+--------------------+--------------------+--------------------+-------------+--------------------+-------+-------------+----------+------+---------+--------------+
|match_id|               date|player_of_match|venue_id|neutral_venue|               team1|               team2|         toss_winner|toss_decision|              winner| result|result_margin|eliminator|method|  umpire1|       umpire2|
+--------+-------------------+---------------+--------+-------------+--------------------+--------------------+--------------------+-------------+--------------------+-------+-------------+----------+------+---------+--------------+
|  335982|2008-04-18 00:00:00|    BB McCullum|      35|            0|Royal Challengers...|Kolkata Knight Ri...|Royal Challengers...|        field|Kolkata Knight Ri...|   runs|          140|         N|    NA|Asad Rauf|   RE Koertzen|
|  335983|2008-04-19 00:00:00|     MEK Hussey|      31|            0

In [ ]:
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df3 = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location3)

df3.show(5)

+--------+--------------------+-------------+
|venue_id|               venue|         city|
+--------+--------------------+-------------+
|       1|Dr. Y.S. Rajasekh...|Visakhapatnam|
|       2|Sharjah Cricket S...|      Sharjah|
|       3|JSCA Internationa...|       Ranchi|
|       4|Saurashtra Cricke...|       Rajkot|
|       5|Shaheed Veer Nara...|       Raipur|
+--------+--------------------+-------------+
only showing top 5 rows



In [ ]:
spark.conf.set("spark.sql.legacy.allowCreatingManagedTableUsingNonemptyLocation","true")

permanent_table_name1 = "ipl_ball_by_ball"
permanent_table_name2 = "ipl_matches"
permanent_table_name3 = "ipl_venue"

df1.write.format("parquet").saveAsTable(permanent_table_name1)
df2.write.format("parquet").saveAsTable(permanent_table_name2)
df3.write.format("parquet").saveAsTable(permanent_table_name3)

In [ ]:
%sql
-- # A. Find the top 3 venues which hosted the most number of eliminator matches?

SELECT b.venue, COUNT(a.eliminator) AS eliminator_count
FROM ipl_matches a
JOIN ipl_venue b
ON a.venue_id == b.venue_id
WHERE a.eliminator = 'Y'
GROUP BY b.venue
LIMIT 3

venue,eliminator_count
Dubai International Cricket Stadium,3
"Sardar Patel Stadium, Motera",1
Feroz Shah Kotla,1


In [ ]:
%sql
-- # B. Return most number of catches taken by a player in IPL history?

SELECT fielder, COUNT(dismissal_kind) AS num_catches 
FROM ipl_ball_by_ball
WHERE fielder != 'NA' AND dismissal_kind = 'caught'
GROUP BY fielder
ORDER BY num_catches DESC
LIMIT 5

fielder,num_catches
KD Karthik,118
MS Dhoni,113
AB de Villiers,103
SK Raina,99
RG Sharma,88


In [ ]:
%sql 
-- # C. Write a query to return a report for highest wicket taker in matches which were affected by Duckworth-Lewis’s method (D/L method).

SELECT a.bowler, COUNT(a.bowler) AS wicket_count 
FROM ipl_ball_by_ball a
JOIN ipl_matches b
ON a.match_id = b.match_id
WHERE b.method = 'D/L' AND a.is_wicket = 1
GROUP BY a.bowler
ORDER BY wicket_count DESC
LIMIT 5

bowler,wicket_count
R Ashwin,8
AB Dinda,8
MC Henriques,7
B Kumar,6
JD Unadkat,5


In [ ]:
%sql
-- # D. Write a query to return a report for highest strike rate by a batsman in non powerplay overs(7-20 overs)
-- # Note: strike rate = (Total Runs scored/Total balls faced by player) *100, Make sure
-- # that balls faced by players should be legal delivery (not wide balls or no balls
-- total batsman runs

SELECT a.batsman, ((a.total_runs/b.balls_faced)*100) AS strike_rate 
FROM (
SELECT batsman, SUM(batsman_runs) AS total_runs FROM ipl_ball_by_ball
WHERE overs >6 
GROUP BY batsman) a
JOIN (
  SELECT batsman, COUNT(batsman) AS balls_faced FROM ipl_ball_by_ball
  WHERE extras_type == 'NA'
  GROUP BY batsman) b
ON a.batsman == b.batsman
ORDER BY strike_rate DESC
LIMIT 10

batsman,strike_rate
B Stanlake,250.0
KMDN Kulasekara,250.0
Umar Gul,229.41176470588235
VRV Singh,200.0
RS Sodhi,200.0
AD Russell,181.71779141104295
TU Deshpande,175.0
I Malhotra,175.0
Abdul Samad,173.4375
BCJ Cutting,172.992700729927


In [ ]:
%sql

-- We can see from the results of previous query; some of the top names in the list did not make sense. That is because sometimes those who face less balls 
-- have high strike rate. In this query the I have set the balls faced parameter to greater than 30. And sure enough the results make much more sense

SELECT a.batsman, ((a.total_runs/b.balls_faced)*100) AS strike_rate 
FROM (
SELECT batsman, SUM(batsman_runs) AS total_runs FROM ipl_ball_by_ball
WHERE overs >6 
GROUP BY batsman) a
JOIN (
  SELECT batsman, COUNT(batsman) AS balls_faced FROM ipl_ball_by_ball
  WHERE extras_type == 'NA'
  GROUP BY batsman
  HAVING balls_faced > 30) b
ON a.batsman == b.batsman
ORDER BY strike_rate DESC
LIMIT 10

batsman,strike_rate
AD Russell,181.71779141104295
Abdul Samad,173.4375
BCJ Cutting,172.992700729927
CR Brathwaite,169.1588785046729
K Gowtham,164.8148148148148
LJ Wright,164.40677966101697
JC Archer,163.8655462184874
HH Pandya,160.53268765133174
KK Cooper,159.0909090909091
Rashid Khan,156.17977528089887


In [ ]:
%sql 

-- # E. Write a query to return a report for highest extra runs in a venue (stadium, city).


SELECT venue, SUM(extra_runs) AS extras
FROM ipl_ball_by_ball a
JOIN ipl_matches b
ON a.match_id == b.match_id
JOIN ipl_venue c
ON b.venue_id == c.venue_id
GROUP BY venue
ORDER BY extras DESC


venue,extras
M.Chinnaswamy Stadium,1355
Wankhede Stadium,1232
Eden Gardens,1215
Feroz Shah Kotla,1114
"MA Chidambaram Stadium, Chepauk",905
"Rajiv Gandhi International Stadium, Uppal",900
Sawai Mansingh Stadium,791
"Punjab Cricket Association Stadium, Mohali",566
Dubai International Cricket Stadium,492
Sheikh Zayed Stadium,387


In [ ]:
%sql 

-- # F. Write a query to return a report for the cricketers with the most number of players of
-- # the match award in neutral venues.

SELECT player_of_match, COUNT(player_of_match) AS count
FROM ipl_matches
WHERE neutral_venue = 1
GROUP BY player_of_match
ORDER BY count DESC
LIMIT 5

player_of_match,count
YK Pathan,3
JH Kallis,3
JP Duminy,3
GJ Maxwell,3
KC Sangakkara,2


In [ ]:
%sql

-- # G. Write a query to get a list of top 10 players with the highest batting average Note:
-- # Batting average is the total number of runs scored divided by the number of times
-- # they have been out (Make sure to include run outs (on non-striker end) as valid
-- # out while calculating average).

SELECT a.batsman, a.runs/b.dismissals AS average
FROM (SELECT batsman, SUM(batsman_runs) AS runs FROM ipl_ball_by_ball
GROUP BY batsman) a
JOIN (SELECT batsman, COUNT(player_dismissed) AS dismissals FROM ipl_ball_by_ball
WHERE player_dismissed != 'NA'
GROUP BY batsman
) b
ON a.batsman == b.batsman
ORDER BY average DESC
LIMIT 10


batsman,average
Iqbal Abdulla,88.0
RD Gaikwad,68.0
MN van Wyk,55.666666666666664
PD Collingwood,50.75
HM Amla,44.38461538461539
LH Ferguson,44.0
KL Rahul,42.693548387096776
AB de Villiers,42.53508771929825
DA Warner,41.698412698412696
JP Duminy,41.40816326530612


In [ ]:
%sql
-- # H. Write a query to find out who has officiated (as an umpire) the most
-- # number of matches in IPL.

SELECT a.umpire1 AS umpire, (a.count1 + b.count2) AS total
FROM (SELECT umpire1, COUNT(umpire1) AS count1 FROM ipl_matches
GROUP BY umpire1) a
JOIN
(SELECT umpire2, count(umpire2) AS count2 FROM ipl_matches
GROUP BY umpire2) b
ON a.umpire1 == b.umpire2
ORDER BY total DESC
LIMIT 1


umpire,total
S Ravi,121


In [ ]:
%sql

-- # I. Find venue details of the match where V Kohli scored his highest individual runs in
-- # IPL.

SELECT DISTINCT d.match_id, d.runs, c.venue
FROM ipl_ball_by_ball a
JOIN ipl_matches b
ON a.match_id == b.match_id
JOIN ipl_venue c
ON b.venue_id == c.venue_id
JOIN (
  SELECT CAST(match_id AS string), sum(batsman_runs) AS runs FROM ipl_ball_by_ball
  WHERE batsman == 'V Kohli'
  GROUP BY match_id
  ORDER BY runs DESC) d
ON a.match_id = d.match_id
ORDER BY runs DESC
LIMIT 5

match_id,runs,venue
980999,113,M.Chinnaswamy Stadium
980987,109,M.Chinnaswamy Stadium
980969,108,M.Chinnaswamy Stadium
1178410,100,Eden Gardens
980937,100,Saurashtra Cricket Association Stadium


In [ ]:
%sql 
-- # J. Creative Case study:
-- # Please analyze how winning/losing tosses can impact a match and it's result?
-- # (Bonus for Visualization here)


SELECT  toss_winner, count(toss_winner) AS match_toss_win
FROM ipl_matches
WHERE toss_winner = winner
GROUP BY toss_winner
ORDER BY match_toss_win DESC;

-- The analysis of this question is divided into multiple cells.
-- (1/n)

toss_winner,match_toss_win
Mumbai Indians,61
Chennai Super Kings,61
Kolkata Knight Riders,55
Rajasthan Royals,44
Royal Challengers Bangalore,43
Kings XI Punjab,36
Delhi Daredevils,35
Sunrisers Hyderabad,29
Deccan Chargers,19
Delhi Capitals,10


In [ ]:
%sql

--  this query shows the teams who have lost the most matches where they won the toss

SELECT  toss_winner, count(toss_winner) AS match_toss_win
FROM ipl_matches
WHERE toss_winner != winner
GROUP BY toss_winner
ORDER BY match_toss_win DESC;

-- (2/n)

toss_winner,match_toss_win
Kings XI Punjab,49
Delhi Daredevils,45
Mumbai Indians,45
Royal Challengers Bangalore,44
Rajasthan Royals,43
Kolkata Knight Riders,43
Chennai Super Kings,36
Sunrisers Hyderabad,28
Deccan Chargers,24
Pune Warriors,17


In [ ]:
%sql

SELECT winner, COUNT(winner) AS wins 
FROM ipl_matches 
GROUP BY winner
ORDER BY wins DESC

-- This graph shows us that the teams who won most with winning toss and the teams who have most overall wins are the same. i.e. Mumbai Indians and CSK. What this 
-- shows us that toss do not have significant effect on match result

-- (3/n)

winner,wins
Mumbai Indians,120
Chennai Super Kings,106
Kolkata Knight Riders,99
Royal Challengers Bangalore,91
Kings XI Punjab,88
Rajasthan Royals,81
Delhi Daredevils,67
Sunrisers Hyderabad,66
Deccan Chargers,29
Delhi Capitals,19


In [ ]:
%sql

SELECT COUNT(winner)
FROM ipl_matches 
WHERE toss_winner = winner

-- This shows how may times the toss winning team has won the match.

-- (4/n)

count(winner)
418


In [ ]:
%sql

SELECT COUNT(winner)
FROM ipl_matches 
WHERE toss_winner != winner

-- This shows how may times the toss winning team has lost the match. As we can see there is difference but it is not very big. We can definately conclude that 
-- toss winning is beneficial but not be all end all for the match

-- (5/5)

count(winner)
398


In [ ]:
import sqlite3
conn = sqlite3.connect("ipl.db")
c = conn.cursor()

In [ ]:
# Converting tables to pandas so they can be loaded into sqlite database

a = sqlContext.sql("select * from ipl_matches").toPandas()
b = sqlContext.sql("select * from ipl_ball_by_ball").toPandas()
d = sqlContext.sql("select * from ipl_venue").toPandas()

In [ ]:
# I will load the data from existing tables to sqlite database

a.to_sql(name = 'ipl_matches', con = conn, schema = a.columns ,if_exists = 'replace')
b.to_sql(name = 'ipl_ball_by_ball', con = conn, schema = b.columns ,if_exists = 'replace')
d.to_sql(name = 'ipl_venue', con = conn, schema = d.columns ,if_exists = 'replace' )

In [ ]:
# Test

c.execute("select a.match_id, a.overs, a.batsman \
from ipl_ball_by_ball a")
c.fetchall()

Out[9]: [(419157, 16, 'V Kohli'),
 (419157, 16, 'V Kohli'),
 (419157, 16, 'R Vinay Kumar'),
 (419157, 17, 'R Vinay Kumar'),
 (419157, 17, 'V Kohli'),
 (419157, 17, 'R Vinay Kumar'),
 (419157, 17, 'R Vinay Kumar'),
 (419157, 17, 'R Vinay Kumar'),
 (419157, 17, 'V Kohli'),
 (419157, 18, 'V Kohli'),
 (419157, 18, 'V Kohli'),
 (419157, 18, 'R Vinay Kumar'),
 (419157, 18, 'R Vinay Kumar'),
 (419157, 18, 'Pankaj Singh'),
 (419157, 18, 'R Vinay Kumar'),
 (419157, 18, 'Pankaj Singh'),
 (419157, 19, 'A Kumble'),
 (419157, 0, 'R McLaren'),
 (419157, 0, 'SR Tendulkar'),
 (419157, 0, 'SR Tendulkar'),
 (419157, 0, 'R McLaren'),
 (419157, 0, 'SR Tendulkar'),
 (419157, 0, 'MK Pandey'),
 (419157, 0, 'MK Pandey'),
 (419157, 0, 'MK Pandey'),
 (419157, 0, 'MK Pandey'),
 (419157, 0, 'MK Pandey'),
 (419157, 0, 'JH Kallis'),
 (419157, 1, 'JH Kallis'),
 (419157, 1, 'JH Kallis'),
 (419157, 1, 'JH Kallis'),
 (419157, 1, 'MK Pandey'),
 (419157, 1, 'MK Pandey'),
 (419157, 1, 'MK Pandey'),
 (419157, 2, 'JH Kallis

In [ ]:
# test

query = '''select a.batsman, a.runs/b.dismissals as average
from (select batsman, SUM(batsman_runs) as runs from ipl_ball_by_ball
group by batsman) a
JOIN (select batsman, count(player_dismissed) as dismissals from ipl_ball_by_ball
where player_dismissed != 'NA'
group by batsman
) b
ON a.batsman == b.batsman
order by average desc
LIMIT 10'''

c.execute(query)
c.fetchall()

Out[10]: [('Iqbal Abdulla', 88),
 ('RD Gaikwad', 68),
 ('MN van Wyk', 55),
 ('PD Collingwood', 50),
 ('HM Amla', 44),
 ('LH Ferguson', 44),
 ('AB de Villiers', 42),
 ('KL Rahul', 42),
 ('CH Gayle', 41),
 ('DA Warner', 41)]

In [ ]:
# test

import json
c.execute(query)
json_output = json.dumps(c.fetchall())
json_output

Out[11]: '[["Iqbal Abdulla", 88], ["RD Gaikwad", 68], ["MN van Wyk", 55], ["PD Collingwood", 50], ["HM Amla", 44], ["LH Ferguson", 44], ["AB de Villiers", 42], ["KL Rahul", 42], ["CH Gayle", 41], ["DA Warner", 41]]'

In [ ]:
import json
# Importing json which will be used to convert results to json object

class Table:
      
    def __init__(self, query):
        '''
        Constructor for the class. Connects to the sqlite databse and receives the query.
        '''
        self.query = query
        
        conn = sqlite3.connect("ipl.db")
        c = conn.cursor()
        
    def get_result(self):
        '''
        Executes the query and returns the result 
        in form of json object.
        '''
        try: 
            c.execute(self.query)
            json_output = json.dumps(c.fetchall())
            return json_output
        except Exception:
              print("Your query did not run")
     
    @staticmethod
    def get_status():
        '''
        Checks the connection to the database. Returns boolean values.
        '''
        try:
            conn.cursor()
            return True
        except Exception as ex:
            return False


In [ ]:
query1 = '''select a.batsman, a.runs/b.dismissals as average
            from (select batsman, SUM(batsman_runs) as runs from ipl_ball_by_ball
            group by batsman) a
            JOIN (select batsman, count(player_dismissed) as dismissals from ipl_ball_by_ball
            where player_dismissed != 'NA'
            group by batsman
            ) b
            ON a.batsman == b.batsman
            order by average desc
            LIMIT 10'''

q1 = Table(query1)

print(q1.get_result())

[["Iqbal Abdulla", 88], ["RD Gaikwad", 68], ["MN van Wyk", 55], ["PD Collingwood", 50], ["HM Amla", 44], ["LH Ferguson", 44], ["AB de Villiers", 42], ["KL Rahul", 42], ["CH Gayle", 41], ["DA Warner", 41]]
